In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from six.moves import urllib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [9]:
df = pd.read_excel('UterineCancerDataBasic.xlsx')

columns_to_drop = ['Sample ID', 'Other Patient ID', 'Form completion date', 'Tissue Source Site',
                   'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code',
                   'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code',
                   'American Joint Committee on Cancer Publication Version Type', 'ICD 10 Classification',
                   'ICD-O-3 Histology Code', 'ICD-O-3 Site Code', 'AJCC Publication Version Type']

# Check if the columns exist in the DataFrame before dropping
columns_to_drop_existing = [col for col in columns_to_drop if col in df.columns]

if columns_to_drop_existing:
    # Drop the specified columns
    df.drop(columns=columns_to_drop_existing, inplace=True)
    
    df.to_excel(output_excel_file, index=False)

identical_columns = [col for col in df.columns if df[col].nunique() == 1]

# Print the names of columns with identical data
print("Columns with identical data:", identical_columns)

# Drop these columns from the DataFrame
df = df.drop(columns=identical_columns)

Columns with identical data: ['Informed consent verified', 'Last Alive Less Initial Pathologic Diagnosis Date Calculated Day Value', 'Neoadjuvant Therapy Type Administered Prior To Resection Text', 'Number of Samples Per Patient', 'Sex', 'TCGA PanCanAtlas Cancer Type Acronym', 'Sample Type', 'Somatic Status', 'Tumor Disease Anatomic Site', 'Cancer Type', 'Targeted Molecular Therapy']


In [10]:
data = df
data.describe()

,Cancer Type Detailed,Overall Survival Status,Disease Free Status,Disease-specific Survival status,Progression Free Status,Mutation Count,Fraction Genome Altered,Diagnosis Age,MSI MANTIS Score,MSIsensor Score,...,Oncotree Code,Tissue Prospective Collection Indicator,Tissue Retrospective Collection Indicator,Tissue Source Site Code,TMB (nonsynonymous),Chemotherapy,Hormone Therapy,Not Specified,Other,Radiation Therapy
count,420.000000,420.000000,420.000000,419.000000,420.000000,407.000000,412.000000,418.000000,417.000000,419.000000,...,420.000000,416.000000,420.000000,420.000000,409.000000,420.000000,420.000000,420.000000,420.000000,420.000000
mean,0.259524,0.109524,0.147619,0.052506,0.164286,1084.646192,0.166886,63.217703,0.438659,6.517733,...,0.259524,0.206731,0.795238,11.109524,37.063570,0.290476,0.028571,0.007143,0.002381,0.438095
std,0.523236,0.312668,0.355145,0.223312,0.370977,2767.489538,0.216792,11.010480,0.241283,10.314635,...,0.523236,0.405448,0.404009,5.599813,95.123871,0.454524,0.166797,0.084313,0.048795,0.496745
min,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,31.000000,0.237000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,47.500000,0.000800,56.000000,0.282100,0.090000,...,0.000000,0.000000,1.000000,6.000000,1.600000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,79.000000,0.049850,63.000000,0.308600,0.430000,...,0.000000,0.000000,1.000000,10.000000,2.800000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,584.500000,0.263750,70.750000,0.543800,10.845000,...,0.000000,0.000000,1.000000,13.000000,19.500000,1.000000,0.000000,0.000000,0.000000,1.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,25696.000000,0.947200,90.000000,1.304900,40.430000,...,2.000000,1.000000,1.000000,29.000000,861.866667,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
columns_with_missing_values = data.columns[data.isnull().any()]

# Print the names of columns with missing values
print("Columns with missing values:")
for col in columns_with_missing_values:
    print(col)

# Optionally, you can also print the number of missing values in each column
print("\nNumber of missing values in each column:")
print(data.isnull().sum())

Columns with missing values:
Disease-specific Survival status
Mutation Count
Fraction Genome Altered
Diagnosis Age
MSI MANTIS Score
MSIsensor Score
Subtype
Birth from Initial Pathologic Diagnosis Date
Buffa Hypoxia Score
Disease Free (Months)
Genetic Ancestry Label
Last Communication Contact from Initial Pathologic Diagnosis Date
New Neoplasm Event Post Initial Therapy Indicator
Patient Weight
Person Neoplasm Cancer Status
Radiation Therapy_x
Ragnum Hypoxia Score
Winter Hypoxia Score
Aneuploidy Score
Tissue Prospective Collection Indicator
TMB (nonsynonymous)

Number of missing values in each column:
Patient ID                                                             0
Cancer Type Detailed                                                   0
Overall Survival Status                                                0
Disease Free Status                                                    0
Disease-specific Survival status                                       1
Progression Free Status    

In [12]:
data = data.dropna(subset=['Disease Free Status'])
print(data.shape[0])
print(data.shape[1])
data_cleaned = data.dropna(axis=1, how='any')
print(data_cleaned.shape[0])
print(data_cleaned.shape[1])
data_shuffled = data_cleaned.sample(frac=1, random_state=42)


# Features (X) are all columns except 'ID' and 'Stage'
X = data_shuffled.drop(['Patient ID', 'Disease Free Status'], axis=1)

# Target variable (y) is 'Stage'
y = data_shuffled['Disease Free Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

420
44
420
23


In [13]:
data = df
data.describe()

,Cancer Type Detailed,Overall Survival Status,Disease Free Status,Disease-specific Survival status,Progression Free Status,Mutation Count,Fraction Genome Altered,Diagnosis Age,MSI MANTIS Score,MSIsensor Score,...,Oncotree Code,Tissue Prospective Collection Indicator,Tissue Retrospective Collection Indicator,Tissue Source Site Code,TMB (nonsynonymous),Chemotherapy,Hormone Therapy,Not Specified,Other,Radiation Therapy
count,420.000000,420.000000,420.000000,419.000000,420.000000,407.000000,412.000000,418.000000,417.000000,419.000000,...,420.000000,416.000000,420.000000,420.000000,409.000000,420.000000,420.000000,420.000000,420.000000,420.000000
mean,0.259524,0.109524,0.147619,0.052506,0.164286,1084.646192,0.166886,63.217703,0.438659,6.517733,...,0.259524,0.206731,0.795238,11.109524,37.063570,0.290476,0.028571,0.007143,0.002381,0.438095
std,0.523236,0.312668,0.355145,0.223312,0.370977,2767.489538,0.216792,11.010480,0.241283,10.314635,...,0.523236,0.405448,0.404009,5.599813,95.123871,0.454524,0.166797,0.084313,0.048795,0.496745
min,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,31.000000,0.237000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,47.500000,0.000800,56.000000,0.282100,0.090000,...,0.000000,0.000000,1.000000,6.000000,1.600000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,79.000000,0.049850,63.000000,0.308600,0.430000,...,0.000000,0.000000,1.000000,10.000000,2.800000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,584.500000,0.263750,70.750000,0.543800,10.845000,...,0.000000,0.000000,1.000000,13.000000,19.500000,1.000000,0.000000,0.000000,0.000000,1.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,25696.000000,0.947200,90.000000,1.304900,40.430000,...,2.000000,1.000000,1.000000,29.000000,861.866667,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
classifier_dt = DecisionTreeClassifier(random_state=42)
classifier_hgb = HistGradientBoostingClassifier(random_state=42)
classifier_mnb = MultinomialNB()
classifier_knn = KNeighborsClassifier()
classifier_gb = GradientBoostingClassifier(random_state=42)

In [15]:
ensemble_classifier = VotingClassifier(
    estimators=[
        ('dt', classifier_dt),
        ('hgb', classifier_hgb),
        ('mnb', classifier_mnb),
        ('knn', classifier_knn),
        ('gb', classifier_gb)
    ],
    voting='soft' 
)

In [16]:
for classifier in (classifier_dt, classifier_hgb, classifier_mnb, classifier_knn, 
                   classifier_gb, 
                   ensemble_classifier):
    classifier.fit(X_train, y_train)

In [17]:
y_pred_dt = classifier_dt.predict(X_test)
y_pred_hgb = classifier_hgb.predict(X_test)
y_pred_mnb = classifier_mnb.predict(X_test)
y_pred_knn = classifier_knn.predict(X_test)
y_pred_gb = classifier_gb.predict(X_test)
y_pred_ensemble = ensemble_classifier.predict(X_test)

In [23]:
for name, y_pred in zip(('Decision Tree', 'Hist Gradient Boosting', 'Multinomial Naive Bayes', 'k-NN', 
                         'Gradient Boosting', 
                         'Ensemble'), 
                        (y_pred_dt, y_pred_hgb, y_pred_mnb, y_pred_knn, 
                         y_pred_gb, 
                         y_pred_ensemble)):
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")
    print(f"{name} Classification Report:\n{classification_report(y_test, y_pred)}")
    print("----------------------------------------------------------------------------------\n")

Decision Tree Accuracy: 0.9881
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        73
           1       1.00      0.91      0.95        11

    accuracy                           0.99        84
   macro avg       0.99      0.95      0.97        84
weighted avg       0.99      0.99      0.99        84

----------------------------------------------------------------------------------

Hist Gradient Boosting Accuracy: 0.9881
Hist Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        73
           1       1.00      0.91      0.95        11

    accuracy                           0.99        84
   macro avg       0.99      0.95      0.97        84
weighted avg       0.99      0.99      0.99        84

----------------------------------------------------------------------------------

Multinomial Naive Bayes 

In [24]:
# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f"\nEnsemble Accuracy: {ensemble_accuracy:.4f}")


Ensemble Accuracy: 1.0000


In [29]:
from sklearn.metrics import confusion_matrix

for name, y_pred in zip(('Decision Tree', 'Hist Gradient Boosting', 'Multinomial Naive Bayes', 'k-NN', 
                         'Gradient Boosting', 
                         'Ensemble'), 
                        (y_pred_dt, y_pred_hgb, y_pred_mnb, y_pred_knn, 
                         y_pred_gb, 
                         y_pred_ensemble)):
    
    # Get the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    print(f"{name} Confusion Matrix:\n{cm}")
    print(f"{name} Sensitivity: {sensitivity:.4f}")
    print(f"{name} Specificity: {specificity:.4f}")
    print("----------------------------------------------------------------------------------\n")

Decision Tree Confusion Matrix:
[[73  0]
 [ 1 10]]
Decision Tree Sensitivity: 0.9091
Decision Tree Specificity: 1.0000
----------------------------------------------------------------------------------

Hist Gradient Boosting Confusion Matrix:
[[73  0]
 [ 1 10]]
Hist Gradient Boosting Sensitivity: 0.9091
Hist Gradient Boosting Specificity: 1.0000
----------------------------------------------------------------------------------

Multinomial Naive Bayes Confusion Matrix:
[[72  1]
 [ 1 10]]
Multinomial Naive Bayes Sensitivity: 0.9091
Multinomial Naive Bayes Specificity: 0.9863
----------------------------------------------------------------------------------

k-NN Confusion Matrix:
[[73  0]
 [ 5  6]]
k-NN Sensitivity: 0.5455
k-NN Specificity: 1.0000
----------------------------------------------------------------------------------

Gradient Boosting Confusion Matrix:
[[73  0]
 [ 0 11]]
Gradient Boosting Sensitivity: 1.0000
Gradient Boosting Specificity: 1.0000
---------------------------

In [36]:
from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report,
                             precision_score, recall_score, f1_score)
from sklearn.model_selection import cross_val_score
import numpy as np
import scipy.stats as stats


# To store the results for statistical tests later
results = {name: {'accuracy': [], 'precision': [], 'recall': [], 'specificity': [], 'f1': []} for name in model_names}

for name, y_pred in zip(('Decision Tree', 'Hist Gradient Boosting', 'Multinomial Naive Bayes', 'k-NN', 
                         'Gradient Boosting', 
                         'Ensemble'), 
                        (y_pred_dt, y_pred_hgb, y_pred_mnb, y_pred_knn, 
                         y_pred_gb, 
                         y_pred_ensemble)):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # Get the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate specificity
    specificity = tn / (tn + fp)
    
    # Calculate F1-score
    f1 = f1_score(y_test, y_pred)
    
    # Store results for statistical analysis
    results[name]['accuracy'].append(accuracy)
    results[name]['precision'].append(precision)
    results[name]['recall'].append(recall)
    results[name]['specificity'].append(specificity)
    results[name]['f1'].append(f1)
    
    print(f"{name} Precision: {precision:.4f}")
    print(f"{name} Recall (Sensitivity): {recall:.4f}")
    print(f"{name} Specificity: {specificity:.4f}")
    print(f"{name} F1-score: {f1:.4f}")
    print("----------------------------------------------------------------------------------\n")
    
# Cross-validation to evaluate model stability
for name, model in zip(('Decision Tree', 'Hist Gradient Boosting', 'Multinomial Naive Bayes', 'k-NN', 
                         'Gradient Boosting', 
                         'Ensemble'), 
                        (classifier_dt, classifier_hgb, classifier_mnb, classifier_knn, 
                         classifier_gb, 
                         ensemble_classifier)):
    cv_scores = cross_val_score(model, X, y, cv=5)  # 5-fold cross-validation
    print(f"{name} Cross-Validation Scores: {cv_scores}")
    print(f"{name} Mean CV Score: {np.mean(cv_scores):.4f}")
    print(f"{name} Standard Deviation of CV Scores: {np.std(cv_scores):.4f}")

# Statistical tests to compare models
# Example: Paired t-test between Decision Tree and Hist Gradient Boosting models
t_stat, p_value = stats.ttest_rel(results['Decision Tree']['accuracy'], results['Hist Gradient Boosting']['accuracy'])
print(f"Paired t-test between Decision Tree and Hist Gradient Boosting accuracies: t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")


Decision Tree Precision: 1.0000
Decision Tree Recall (Sensitivity): 0.9091
Decision Tree Specificity: 1.0000
Decision Tree F1-score: 0.9524
----------------------------------------------------------------------------------

Hist Gradient Boosting Precision: 1.0000
Hist Gradient Boosting Recall (Sensitivity): 0.9091
Hist Gradient Boosting Specificity: 1.0000
Hist Gradient Boosting F1-score: 0.9524
----------------------------------------------------------------------------------

Multinomial Naive Bayes Precision: 0.9091
Multinomial Naive Bayes Recall (Sensitivity): 0.9091
Multinomial Naive Bayes Specificity: 0.9863
Multinomial Naive Bayes F1-score: 0.9091
----------------------------------------------------------------------------------

k-NN Precision: 1.0000
k-NN Recall (Sensitivity): 0.5455
k-NN Specificity: 1.0000
k-NN F1-score: 0.7059
----------------------------------------------------------------------------------

Gradient Boosting Precision: 1.0000
Gradient Boosting Recall (Se